<a href="https://colab.research.google.com/github/mohsenfayyaz/NLP_Assignments/blob/master/Week2_Recognize_named_entities_on_Twitter_with_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week2()

import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

--2020-07-21 17:14:49--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2020-07-21 17:14:49 (19.7 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]



In [ ]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            if token[0] == "@":
                token = "<USR>"
            if "http://" in token or "https://" in token:
                token = "<URL>"

            ######################################
            ######### YOUR CODE HERE #############
            ######################################
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [ ]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [ ]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



**Prepare dictionaries**

In [ ]:
from collections import defaultdict

In [ ]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    # At first, add special tokens (or tags) to the dictionaries.
    # The first special token must have index 0.
    idx2tok = special_tokens
    for index, token in enumerate(idx2tok):
        tok2idx[token] = index


    # Mapping tok2idx should contain each token or tag only once.
    # To do so, you should:
    # 1. extract unique tokens/tags from the tokens_or_tags variable, which is not
    #    occur in special_tokens (because they could have non-empty intersection)
    # 2. index them (for example, you can add them into the list idx2tok
    # 3. for each token/tag save the index into tok2idx).
    for sentence in tokens_or_tags:
        for token in sentence:
            if token not in idx2tok:
                tok2idx[token] = len(idx2tok)
                idx2tok.append(token)

    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    return tok2idx, idx2tok

In [ ]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

print(token2idx)

defaultdict(<function build_dict.<locals>.<lambda> at 0x7f299a9377b8>, {'<UNK>': 0, '<PAD>': 1, 'RT': 2, '<USR>': 3, ':': 4, 'Online': 5, 'ticket': 6, 'sales': 7, 'for': 8, 'Ghostland': 9, 'Observatory': 10, 'extended': 11, 'until': 12, '6': 13, 'PM': 14, 'EST': 15, 'due': 16, 'to': 17, 'high': 18, 'demand': 19, '.': 20, 'Get': 21, 'them': 22, 'before': 23, 'they': 24, 'sell': 25, 'out': 26, '...': 27, 'Apple': 28, 'MacBook': 29, 'Pro': 30, 'A1278': 31, '13.3': 32, '"': 33, 'Laptop': 34, '-': 35, 'MD101LL/A': 36, '(': 37, 'June': 38, ',': 39, '2012': 40, ')': 41, 'Full': 42, 'read': 43, 'by': 44, 'eBay': 45, '<URL>': 46, 'Happy': 47, 'Birthday': 48, '!': 49, 'May': 50, 'Allah': 51, 's.w.t': 52, 'bless': 53, 'you': 54, 'with': 55, 'goodness': 56, 'and': 57, 'happiness': 58, 'the': 59, 'quest': 60, 'line': 61, 'im': 62, 'assuming': 63, 'it': 64, 'will': 65, 'be': 66, 'same': 67, 'way': 68, 'awe': 69, 'thur': 70, 'still': 71, 'perception': 72, 'what': 73, 'key': 74, 'off': 75, 'may': 76, 

In [ ]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

**Generate batches**

In [ ]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

**Build a recurrent neural network**

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import numpy as np

TensorFlow 1.x selected.
1.15.2


In [ ]:
class BiLSTMModel():
    pass


*  input_batch — sequences of words (the shape equals to [batch_size, sequence_len)

*  ground_truth_tags — sequences of tags (the shape equals to [batch_size, sequence_len]);

*  lengths — lengths of not padded sequences (the shape equals to [batch_size]);

*  dropout_ph — dropout keep probability; this placeholder has a predefined value 1;

*  learning_rate_ph — learning rate; we need this placeholder because we want to change the value during training.


In [ ]:
def declare_placeholders(self):
    """Specifies placeholders for the model."""

    # Placeholders for input and ground truth output.
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')  ######### YOUR CODE HERE #############
  
    # Placeholder for lengths of the sequences.
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    # Placeholder for a dropout keep probability. If we don't feed
    # a value for this placeholder, it will be equal to 1.0.
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    # Placeholder for a learning rate (tf.float32).
    self.learning_rate_ph = tf.placeholder(dtype=tf.float32, name='learning_rate_ph')  ######### YOUR CODE HERE #############

In [ ]:
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [ ]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """Specifies bi-LSTM architecture and computes logits for inputs."""
    
    # Create embedding variable (tf.Variable) with dtype tf.float32
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(dtype=tf.float32, initial_value=initial_embedding_matrix, name="embeddings_matrix")
  ######### YOUR CODE HERE #############
    
    # Create RNN cells (for example, tf.nn.rnn_cell.BasicLSTMCell) with n_hidden_rnn number of units 
    # and dropout (tf.nn.rnn_cell.DropoutWrapper), initializing all *_keep_prob with dropout placeholder.
    forward_cell =  tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units = n_hidden_rnn), state_keep_prob=self.dropout_ph, input_keep_prob=self.dropout_ph, output_keep_prob=self.dropout_ph) ######### YOUR CODE HERE #############
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units = n_hidden_rnn), state_keep_prob=self.dropout_ph, input_keep_prob=self.dropout_ph, output_keep_prob=self.dropout_ph) ######### YOUR CODE HERE #############

    # Look up embeddings for self.input_batch (tf.nn.embedding_lookup).
    # Shape: [batch_size, sequence_len, embedding_dim].
    embeddings = tf.nn.embedding_lookup(params=embedding_matrix_variable, ids=self.input_batch, name=None) ######### YOUR CODE HERE #############
    
    # Pass them through Bidirectional Dynamic RNN (tf.nn.bidirectional_dynamic_rnn).
    # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
    # Also don't forget to initialize sequence_length as self.lengths and dtype as tf.float32.
    (rnn_output_fw, rnn_output_bw), _ =  tf.nn.bidirectional_dynamic_rnn(cell_fw=forward_cell, cell_bw=backward_cell, inputs=embeddings, sequence_length=self.lengths, dtype=tf.float32)  ######### YOUR CODE HERE #############
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # Dense layer on top.
    # Shape: [batch_size, sequence_len, n_tags].   
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [ ]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [ ]:
def compute_predictions(self):
    """Transforms logits to probabilities and finds the most probable tags."""
    
    # Create softmax (tf.nn.softmax) function
    softmax_output = tf.nn.softmax(logits = self.logits, axis=None, name=None)  ######### YOUR CODE HERE #############
    
    # Use argmax (tf.argmax) to get the most probable tags
    # Don't forget to set axis=-1
    # otherwise argmax will be calculated in a wrong way
    self.predictions = tf.math.argmax(input=softmax_output, axis=-1, output_type=tf.dtypes.int64, name=None)  ######### YOUR CODE HERE #############

In [ ]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [ ]:
def compute_loss(self, n_tags, PAD_index):
    """Computes masked cross-entopy loss with logits."""
    
    # Create cross entropy function function (tf.nn.softmax_cross_entropy_with_logits_v2)
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=ground_truth_tags_one_hot)  ######### YOUR CODE HERE #############
    
    mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
    # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
    # Be careful that the argument of tf.reduce_mean should be
    # multiplication of mask and loss_tensor.
    self.loss = tf.reduce_mean(tf.multiply(mask, loss_tensor))  ######### YOUR CODE HERE #############

In [ ]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [ ]:
def perform_optimization(self):
    """Specifies the optimizer and train_op for the model."""
    
    # Create an optimizer (tf.train.AdamOptimizer)
    self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate_ph, name='Adam')  ######### YOUR CODE HERE #############
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
    # Pay attention that you need to apply this operation only for gradients 
    # because self.grads_and_vars also contains variables.
    # list comprehension might be useful in this case.
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars = [(tf.clip_by_norm(t=g, clip_norm=clip_norm), v) for g, v in self.grads_and_vars]  ######### YOUR CODE HERE #############
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [ ]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [ ]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [ ]:
BiLSTMModel.__init__ = classmethod(init_model)

**Train the network and predict tags**

In [ ]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [ ]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)



Implement the function predict_for_batch by initializing feed_dict with input x_batch and lengths and running the session for self.predictions.


In [ ]:
def predict_for_batch(self, session, x_batch, lengths):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    feed_dict = {self.input_batch: x_batch,
                 self.lengths: lengths}
    
    predictions = session.run(fetches=self.predictions, feed_dict=feed_dict, options=None, run_metadata=None)

    return predictions

In [ ]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

**Evaluation**

In [ ]:
from evaluation import precision_recall_f1

In [ ]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

**Run your experiment**



Create BiLSTMModel model with the following parameters:

    vocabulary_size — number of tokens;
    n_tags — number of tags;
    embedding_dim — dimension of embeddings, recommended value: 200;
    n_hidden_rnn — size of hidden layers for RNN, recommended value: 200;
    PAD_index — an index of the padding token (<PAD>).

Set hyperparameters. You might want to start with the following recommended values:

    batch_size: 32;
    4 epochs;
    starting value of learning_rate: 0.005
    learning_rate_decay: a square root of 2;
    dropout_keep_probability: try several values: 0.1, 0.5, 0.9.

However, feel free to conduct more experiments to tune hyperparameters and earn extra points for the assignment.


In [ ]:
tf.reset_default_graph()

model = BiLSTMModel(
    vocabulary_size=len(token2idx),
    n_tags=len(tag2idx),
    embedding_dim=200,
    n_hidden_rnn=200,
    PAD_index=token2idx['<PAD>']
) ######### YOUR CODE HERE #############

batch_size = 32  ######### YOUR CODE HERE #############
n_epochs = 5  ######### YOUR CODE HERE #############
learning_rate = 0.005  ######### YOUR CODE HERE #############
learning_rate_decay = np.sqrt(2)######### YOUR CODE HERE #############
dropout_keep_probability = 0.5######### YOUR CODE HERE #############

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 76151 phrases; correct: 175.

precision:  0.23%; recall:  3.90%; F1:  0.43

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 9126 phrases; correct: 13.

precision:  0.14%; recall:  2.42%; F1:  0.27

-------------------- Epoch 2 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 1147 phrases; correct: 306.

precision:  26.68%; recall:  6.82%; F1:  10.86

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 101 phrases; correct: 29.

precision:  28.71%; recall:  5.40%; F1:  9.09

-------------------- Epoch 3 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4037 phrases; correct: 1700.

precision:  42.11%; recall:  37.87%; F1:  39.88

Validation data evaluation:
processed 12836 tokens with 537 phra

In [44]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens, validation_tags, short_report=False)  ######### YOUR CODE HERE #############

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False)  ######### YOUR CODE HERE #############

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4661 phrases; correct: 3925.

precision:  84.21%; recall:  87.44%; F1:  85.79

	     company: precision:   88.76%; recall:   93.31%; F1:   90.98; predicted:   676

	    facility: precision:   79.01%; recall:   86.31%; F1:   82.50; predicted:   343

	     geo-loc: precision:   90.72%; recall:   96.18%; F1:   93.37; predicted:  1056

	       movie: precision:   36.36%; recall:   17.65%; F1:   23.76; predicted:    33

	 musicartist: precision:   65.70%; recall:   68.53%; F1:   67.09; predicted:   242

	       other: precision:   82.27%; recall:   90.09%; F1:   86.00; predicted:   829

	      person: precision:   89.33%; recall:   95.49%; F1:   92.31; predicted:   947

	     product: precision:   71.73%; recall:   84.59%; F1:   77.63; predicted:   375

	  sportsteam: precision:   81.41%; recall:   58.53%; F1:   68.10; predicted:   156

	      tvshow: precision:   25.00%; recall:  